## Extraktionsbaserad textsammanfattare med olika rankningsmått 

#### Imports

In [1]:
# Imports --> TODO fixa ordning 
# Web scrapping -->  module för att ladda ner artiklar 
from newspaper import Article
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk 
import ssl
import re  # används ej någonstans (tror jag) / björn
import regex as rex # 


# Summarization length of original text
percentage = 0.15
n_sentences = 3

# Fixes some errors, found online at https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#### Input

## Lägg till nya artiklar 

* Hämta komplett url 
* Lägg till i python cellen under här
* Tilldela till variabeln text 
* Tryck Restart och sen Run All 
* Filerna hamnar i mappen "summarizations"

In [2]:
#text = "https://www.aftonbladet.se/nyheter/a/kE6ExL/sd-far-tunga-poster-i-utskotten"
#text = 'https://www.svt.se/nyheter/utrikes/stall-dina-fragor-om-kriget-till-svt-s-utrikesreportrar'
#text = "https://www.aftonbladet.se/nyheter/a/kE6j0a/uppgifter-viktigt-fynd-i-jakten-pa-mordarna-i-sodertalje"
text = 'https://www.svt.se/nyheter/utrikes/foi-expert-varnar-for-allvarliga-luckor-i-undervattensbevakningen'

In [3]:
article = Article(text, language='sv')
article.download()
article.parse()
text = article.text
homepage = article.meta_data['al']['ios']['app_name']
link = article.url
title = article.title

# Beroende på vilken hemsida nyheten kommer ifrån kan titeln och texten inehålla delar av sidan man egentligen inte bryr sig om
# T.ex. från aftonbladet är titeln med i texten och texten innehåller en mening som: "publicerad: 30 sep", man kan ta bort detta men 
# det blir om vi får tid över.

#print('Title:' , article.title, '\n\nText: \n', text)

# Preprocessing 

## Overview
### Calculate number of sentences to keep

### List 1 - sentences
* Varje mening separat

### Dataframe - scores
#### columns are the score of each ranking measure
* Baseline
* Headings
* TF/IDF-score
* NER
* ~~Class~~ //Om vi har tid för ML

### List 2 -> Cleaned for Stop Words 
* Varje mening separat 

###

##### Original Sentences List

In [4]:
# removes endlines:
from token import NEWLINE

org_sentences = text.replace('\n\n', '. ')
# creates some exceptions from above rule
org_sentences = org_sentences.replace('.. ', '. ')
org_sentences = org_sentences.replace(':. ', ': ')
org_sentences = org_sentences.split('. ')

sentences_to_remove = []
for i, sentence in enumerate(org_sentences): 
    if sentence == title: 
        sentences_to_remove.append(sentence)
    if 'publicerad:' in sentence.lower(): 
        sentences_to_remove.append(sentence)

org_sentences = [sentence for sentence in org_sentences if sentence not in sentences_to_remove]

org_sentences[0:5]

['INSÄNDARE',
 'Socialdemokraternas partiledare Magdalena Andersson kallar till presskonferens för att meddela ett historiskt beslut: ”Det Socialdemokratiska arbetarpartiet har nu kommit till vägs ände, vi har fullgjort vårt uppdrag med att göra rättvisa för och lyfta det arbetande folket',
 'Vi låter nu andra politiska krafter ta vid, som är mer lämpade att lösa 2000-talets problem',
 'Vi socialdemokrater avvecklar nu vårt politiska parti och tackar samtidigt för hundra år av förtroende.”',
 '']

##### Dataframe

In [5]:
index = range(org_sentences.__len__())
columns = ['Baseline', 'Headings', 'TF', 'NER']
scores = pd.DataFrame(index=index, columns=columns)
scores.fillna(0, inplace=True)
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Baseline  28 non-null     int64
 1   Headings  28 non-null     int64
 2   TF        28 non-null     int64
 3   NER       28 non-null     int64
dtypes: int64(4)
memory usage: 1.0 KB


#### Create spacy doc object

In [6]:
import spacy
# Credit to Explosion for sv_core_news_sm --> https://github.com/explosion 
# "lemmatization accuracy 0.95"
# Create spacy nlp object 
nlp = spacy.load("sv_core_news_sm") # nlp used by lemmatizer()

#### Lemmatizer

In [7]:
# den --> det 
# noterar detta att det är en dålig lemmatiserare --> språkbanken stanza / lemmy / kth...  

def lemmatizer(list_of_strings):
    lemmatized_sentences = []
    lemmatized_sentence = ''
    for i in range(len(list_of_strings)): 
        sentence_to_lemmatize = nlp(list_of_strings[i])
        for token in sentence_to_lemmatize:
            lemma = token.lemma_
            lemmatized_sentence += lemma + ' '  
        
        lemmatized_sentences.append(lemmatized_sentence)
        lemmatized_sentence = '' 

    return lemmatized_sentences

# Created Lemmatized DS
lemmatized_org_sentences = lemmatizer(org_sentences)
print(lemmatized_org_sentences)

['INSÄNDARE ', 'Socialdemokraterna partiledare Magdalena Andersson kalla till presskonfer för att meddela en historisk beslut : ” en socialdemokratisk arbetarparti ha nu komma till väga ände , vi ha fullgöra vi uppdrag med att göra rättvisa för och lyfta en arbeta folk ', 'vi låta nu annan politisk kraft ta vid , som vara mycket lämpa att lösa 2000-tal problem ', 'vi socialdemokrat avveckl nu vi politisk parti och tacka samtidig för hundra år av förtroende . ” ', '', 'denna vara vad som böra hända för cirka 15 år sedan , men aldrig komma att hända ', 'anledning vara att Socialdemokrater inte länge vara en politisk parti , den vara en maktstruktur ', 'man ha systematisk under 100 år skapa organisation och en föreningsliv som finnas närvarande i varje liten vrå av samhälle , LO , Folkets hus & park , Verdandi , PRO , lista kunna göra meterlång och denna garantera en marinerande av folk i en självdestruktiv värdegrund som idag prägla Sverige ', 'för hundra år sedan vara S en led i en uppr

#### Proper Nouns

In [8]:
# För bättre täckning på NER 
def proper_nouns(list_of_strings):
    proper_nouns = set()
    for i in range(len(list_of_strings)): 
        sentence_to_pos = nlp(list_of_strings[i])
        for token in sentence_to_pos: 
            token_str = token.text
            if token.pos_ == "PROPN" and len(token_str) > 1:
                proper_nouns.add(token_str.strip())
    return proper_nouns

print(proper_nouns(org_sentences))


{'Jizyah', 'Socialdemokraternas', 'Park', 'LO', 'PRO', 'Olsson', 'Lena', 'Sverige', 'Verdandi', 'Östhammar', 'Mellanöstern', 'Europa', 'Afrika', 'Mats', 'Andersson'}


#### Named Entities

In [9]:
def named_entity_recognition(list_of_strings):
    doc = nlp(' '.join(list_of_strings))
    # Convert tuple[Span] to str
    named_entities = doc.ents.__str__()
    # Remove string parenthesis 
    named_entities = named_entities[1:len(named_entities) - 1]
    # Create list of strings
    named_entities = named_entities.split(',')
    
    named_entities_set = set()
    for entity in named_entities: 
        named_entities_set.add(entity.strip())
    return named_entities_set

### Stop Word Filtering

In [10]:
# Inspired by https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def get_swe_stop_words(): 
    swe_stop_words = set(stopwords.words('swedish'))
    # Not stop word cleaning 
    swe_stop_words.update([',', '"', ':', '-', '–', '”'])
    return swe_stop_words

def stop_word_filtering(list_of_strings):
    word_tokens = word_tokenize(' '.join(list_of_strings))
    filtered_sentence_into_singletons = [w for w in word_tokens if not w.lower() in get_swe_stop_words()]
    return filtered_sentence_into_singletons

#### Frequency Distribution 

In [11]:
def frequency_distribution(list): 
    fdist = FreqDist(word.lower() for word in word_tokenize(' '.join(list)))
    return fdist

In [12]:
filtered_words = stop_word_filtering(lemmatized_org_sentences) 
fdist = frequency_distribution(filtered_words)

# Ranking Measures

### Baseline
(1/N --> n=ordning mening kommer i dvs. första meningen får N=1 -> 1/1, andra meningen får N=2 -> 1/2, osv.)

In [13]:
# Ranking metric 1 --> Baseline
for i, score in enumerate(scores['Baseline']) :
    scores['Baseline'][i] = 1/((i+1))
scores.describe()

,Baseline,Headings,TF,NER
count,28.000000,28.0,28.0,28.0
mean,0.140256,0.0,0.0,0.0
std,0.198050,0.0,0.0,0.0
min,0.035714,0.0,0.0,0.0
25%,0.047078,0.0,0.0,0.0
50%,0.069048,0.0,0.0,0.0
75%,0.129464,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0


### Headings

In [14]:
# Ranking metric 2 --> Headings
# Sets all 'Headings' scores to 0, mostly for testing so i can run this multiple times, 
# but also to make sure nothing weird has happened earlier in the code.
scores['Headings'] = 0
for i, sentence in enumerate(org_sentences): # Lemmatizerad? 
    for word in article.title.split(' '):
        if word in sentence:
            scores.at[i, 'Headings'] += 1
scores.describe()

,Baseline,Headings,TF,NER
count,28.000000,28.000000,28.0,28.0
mean,0.140256,1.071429,0.0,0.0
std,0.198050,1.303232,0.0,0.0
min,0.035714,0.000000,0.0,0.0
25%,0.047078,0.000000,0.0,0.0
50%,0.069048,1.000000,0.0,0.0
75%,0.129464,2.000000,0.0,0.0
max,1.000000,5.000000,0.0,0.0


Sverigedemokraterna få ordförandepost i riksdag justitie och utrikesutskott 
[' få ', 'ordförandepost ', 'riksdag ', 'utrikesutskott ']

* bara ord i lemma 
* ord i lemma med efter letters
* ord i lemma med före letters --> x
* före + lemma + efter 

In [15]:
preprocessed_title = stop_word_filtering(lemmatizer(title.split(' ')))
p = rex.compile(r"\b\S*\L<words>\S*\b | \b\L<words>\b | \b\L<words>\S*\b", re.IGNORECASE, words=preprocessed_title)
p.findall

for i, sentence in enumerate(lemmatized_org_sentences):    
    #print(sentence)
    matches = p.findall(sentence)
    #print(matches)
    scores.at[i, "Headings"] = len(matches)

scores.describe()

,Baseline,Headings,TF,NER
count,28.000000,28.000000,28.0,28.0
mean,0.140256,0.214286,0.0,0.0
std,0.198050,0.686221,0.0,0.0
min,0.035714,0.000000,0.0,0.0
25%,0.047078,0.000000,0.0,0.0
50%,0.069048,0.000000,0.0,0.0
75%,0.129464,0.000000,0.0,0.0
max,1.000000,3.000000,0.0,0.0


## TERM FREQUENCY 


In [16]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
#Term Frequency
def termfreq(sentence, word):
    N = len(sentence)
    occurance = len([token for token in sentence if token == word])
    return occurance/N

In [17]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
#Inverse Document Frequency
def inverse_doc_freq(word):
    try:
        word_occurance = fdist.get(word) + 1
    except:
        word_occurance = 1
    return np.log(len(lemmatized_org_sentences)/word_occurance)

In [18]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
# TF*IDF-score 
def tf_idf(sentence):
    score = 0
    #tf_idf_vec = np.zeros((len(filtered_words),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)
         
        score += tf*idf
        #tf_idf_vec[index_dict[word]] = value 
    return score

for i, sentence in enumerate(lemmatized_org_sentences):
    scores.at[i, 'TF'] = tf_idf(sentence)

#scores.describe()
#scores['TF'].idxmax()
#scores.loc[scores['TF'] == scores['TF'].median()]

### TF*IDF-score

https://forketyfork.medium.com/latex-math-formulas-a-cheat-sheet-21e5eca70aae

In [19]:
# Ranking metric 3 --> TF*IDF

# Term Weights --> Calculate importance of single words in text/doc
# Binary term weights --> document specific
# TF*IDF term weights --> document-collection specific 

# Assign weights to each dimension (attr/word) of each sentence (record/example) 

# Term Frequency (TF-score) --> TFij == frequency of the jth term in in the ith doc 

# Inverse Document Frequency 
# idf-score of the jth term measures the uniqueness of the jth term in the collection of documents
# IDFj = log(M / Nj)
#
# M = total num of docs in collection 
# Nj is the number of documents that contain the jth term

# HIGH TF*IDF-score 
# Word frequent in document && Occur in few documents of the collection 
# LOW TF*IDF-score
# Not present in document || present in all documents of the collection 

### NER 

In [20]:
scores['NER'] = 0
named_entities = named_entity_recognition(org_sentences)
proper_nouns = proper_nouns(org_sentences)
ner_unique = named_entities.union(proper_nouns)
print(ner_unique)

{'Jizyah', 'under 1900-talet', 'PRO', 'LO', 'Magdalena Andersson', 'Lena', 'Östhammar', 'Mellanöstern', 'Europa', 'Afrika', 'idag', 'Socialdemokraternas', 'Park', 'Olsson', 'För hundra år sedan', 'Mats Olsson', 'Sverige', 'Verdandi', 'Mats', 'Andersson'}


In [21]:
p = rex.compile(r"\L<words>", words=ner_unique)

for i, sentence in enumerate(org_sentences):
    matches = p.findall(sentence)
    scores.at[i, "NER"] = len(matches)

scores.describe()
scores.head()


,Baseline,Headings,TF,NER
0,1.000000,0,3.998645,0
1,0.500000,3,59.234508,2
2,0.333333,0,23.779505,0
3,0.250000,1,23.322706,0
4,0.200000,0,0.000000,0


# Combination Function

#### Standardize

In [22]:
# Standardize
scores_standardized = StandardScaler.fit_transform(self=StandardScaler(), X=scores)
scores_standardized = pd.DataFrame(scores_standardized, columns=columns)
scores_standardized['Baseline'] *= 1.5
scores_standardized['Headings'] *= 2.5
scores_standardized['TF'] *= 0.3
scores_standardized

# ÄNDRA COL NAME TF col --> TF*IDF TODO 
# COUNTER WEIGHT: 
# IF -->  
# Baseline score low (e.g. below median)
# && term_frequency high (note: not TF*IDF) (e.g. above upper quartile)
# && NER high (e.g. above upper quartile)
# THEN: 
# add_counterweight(sentence, counter_weight) 
# counter_weight:  
# final rankposition = i -->  ∑ -1 (-1 may be too big weight)

,Baseline,Headings,TF,NER
0,6.631063,-0.794998,-0.351159,-0.470231
1,2.774645,10.334979,0.559938,1.078765
2,1.489173,-0.794998,-0.024880,-0.470231
3,0.846437,2.914994,-0.032415,-0.470231
4,0.460795,-0.794998,-0.417115,-0.470231
5,0.203700,-0.794998,-0.003078,-0.470231
6,0.020061,6.624987,0.013622,-0.470231
7,-0.117668,-0.794998,0.710758,4.176756
8,-0.224791,-0.794998,-0.016615,1.853263
9,-0.310489,-0.794998,0.191060,-0.470231


#### Calculate Summarization Length (number of sentences)

In [23]:
num_of_org_sentences = len(org_sentences)
summarization_num_sentences = round(num_of_org_sentences * percentage)

print("summarization: ", summarization_num_sentences, "\noriginal: ", num_of_org_sentences)

summarization:  4 
original:  28


#### Combine
* Combine the scores into one overall score
* add weight and/or ML if time allows

In [24]:
# Combination Function
# Här ligger ML om vi gör det  

#final_score = scores_standardized.drop('TF', axis=1).sum(axis=1)
final_score = scores_standardized.sum(axis=1)
best_sentences = final_score.nlargest(summarization_num_sentences, keep='all').index.values
print(best_sentences)

print(org_sentences[0])


[1 6 0 7]
INSÄNDARE


In [25]:
# Similarity heurstic 
# Rubrik --> Hur mycket lika --> Ta bort redundans 
# 5 fem väldigt lika --> heuristik välja bort något av det     

# Assemble output 
* Reassemble according to overall score ranking
* Output summarization 

In [26]:
# Percentage length of original text --> summarization 
print("Percentage:\n")
print(title, "\n")
for i in best_sentences: 
    print(org_sentences[i])

print("\n\n")

# N sentences 
print("N sentences:\n")
for i in best_sentences[0:n_sentences]: 
    print(org_sentences[i])

Percentage:

Socialdemokraterna – en maktstruktur, inte ett parti 

Socialdemokraternas partiledare Magdalena Andersson kallar till presskonferens för att meddela ett historiskt beslut: ”Det Socialdemokratiska arbetarpartiet har nu kommit till vägs ände, vi har fullgjort vårt uppdrag med att göra rättvisa för och lyfta det arbetande folket
Anledningen är att Socialdemokraterna inte längre är ett politiskt parti, det är en maktstruktur
INSÄNDARE
Man har systematiskt under 100 år skapat organisationer och ett föreningsliv som finns närvarande i varje liten vrå av samhället, LO, Folkets Hus & Park, Verdandi, PRO, listan kan göras meterlång och dessa garanterar ett marinerande av folket i den självdestruktiva värdegrund som idag präglar Sverige



N sentences:

Socialdemokraternas partiledare Magdalena Andersson kallar till presskonferens för att meddela ett historiskt beslut: ”Det Socialdemokratiska arbetarpartiet har nu kommit till vägs ände, vi har fullgjort vårt uppdrag med att göra rä

In [27]:
# Newspaper Summarization
article.nlp()
print("\nNewspaper3k: \n", article.summary)


Newspaper3k: 
 Anledningen är att Socialdemokraterna inte längre är ett politiskt parti, det är en maktstruktur.
Dessa är sällan sekulära utan de flest lyder blint de heliga skrifterna i vilka arbete är underordnat de religiösa riterna och det religiösa levnadsregler som skall utföras under dygnets vakna timmar.
Vi kallar det socialbidrag, flerbarnstillägg, efterlevandeersättning och etableringsersättning.
Muslimerna kallar det ”Jizyah”, den muslimska skyddsskatten som icke troende skall betala enligt koranen.
Man pratar aldrig om att fler chefer inom stat och kommun nu är kvinnor, inga utbildningsvägar eller yrken är stängda för kvinnor, man pratar bara om att näringslivet inte har tillräckligt många kvinnor i styrelserna.


In [28]:
summarization = link + '\n\n' + 'Title: ' + '\n' + title + '\n\n' + 'Summarization:' + '\n'
for sentence in best_sentences: 
    summarization += org_sentences[sentence] + '\n'
summarization += '\n\n' + 'Newspaper3k:' + '\n' + 'Title: ' + '\n' + title + "\n\n" + article.summary

In [29]:
# import os 
# FILEPATH = f'./summarizations/{homepage}/'
# filename = f"{FILEPATH}{title}.txt"
# os.makedirs(os.path.dirname(filename), exist_ok=True)

# with open(filename, 'w') as f:
#      f.write(summarization)